# Homework 4 - Computing Point-in-Time Residual Returns
In this homework, we will use regressions to compute beta-adjusted "residual" returns in a point-in-time fashion suitable for backtesting / live trading.


1. Download Daily Bars for FB, AAPL, AMZN, NFLX, GOOGL and QQQ from yahoo finance starting 2016-01-01. Use the Adj Close to compute daily returns.




In [15]:
import yfinance as yf
import numpy as np
import pandas as pd



tickers = ['META', 'AAPL', 'AMZN', 'NFLX', 'GOOGL', 'QQQ']
df = yf.download(tickers, start="2025-01-01")['Close']

ret = df / df.shift() - 1
ret

[*********************100%***********************]  6 of 6 completed


Ticker,AAPL,AMZN,GOOGL,META,NFLX,QQQ
Date,,,,,,
2025-01-02,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-03,-0.002009,0.018027,0.012458,0.008995,-0.006406,0.016365
2025-01-06,0.006739,0.015255,0.026487,0.042290,0.000840,0.011493
2025-01-07,-0.011388,-0.024164,-0.007010,-0.019534,-0.002949,-0.017844
2025-01-08,0.002023,0.000090,-0.007878,-0.011604,-0.004766,0.000175
...,...,...,...,...,...,...
2025-04-07,-0.036734,0.024912,0.007898,0.022824,0.013986,0.002413
2025-04-08,-0.049818,-0.026247,-0.013969,-0.011235,0.002961,-0.018008
2025-04-09,0.153289,0.119770,0.096821,0.147556,0.086248,0.120031


2. Now, let's compute the beta of FB, AAPL, AMZN, NFLX, GOOGL using QQQ as our benchmark. You can think of this as the beta these stocks have to their industry (tech). In practice,  we have to use some lookback window to compute the beta. Let's use 252 (1 year, excluding wknds/holidays). So, for each day, the betas should be computed using the most recent 252 data points.

2. Now, let's compute the beta of FB, AAPL, AMZN, NFLX, GOOGL using QQQ as our benchmark. You can think of this as the beta these stocks have to their industry (tech). In practice,  we have to use some lookback window to compute the beta. Let's use 252 (1 year, excluding wknds/holidays). So, for each day, the betas should be computed using the most recent 252 data points.

In [14]:
corr = ret.rolling(252).corr(ret['QQQ'])
vol = ret.rolling(252).std()
beta = (corr*vol).divide(vol['QQQ'],axis=0)
beta





Ticker,AAPL,AMZN,GOOGL,META,NFLX,QQQ
Date,,,,,,
2025-01-02,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-03,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-06,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-07,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-08,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-04-07,NaN,NaN,NaN,NaN,NaN,NaN
2025-04-08,NaN,NaN,NaN,NaN,NaN,NaN
2025-04-09,NaN,NaN,NaN,NaN,NaN,NaN





5. Using the betas, compute an "alpha" on each day. This is also known as a "residual return".


In [16]:
resid = ret - beta.multiply(ret['QQQ'], 0)




4. Compare the volatility of the residual returns to that of the original returns. What do you notice?


In [18]:
vol = {}
vol['orig'] = ret.std() * np.sqrt(252)
vol['resid'] = resid.std() * np.sqrt(252)
vol = pd.DataFrame(vol).drop('QQQ')
vol

,orig,resid
Ticker,,
AAPL,0.488023,NaN
AMZN,0.425312,NaN
GOOGL,0.389564,NaN
META,0.490063,NaN
NFLX,0.436902,NaN




5. Compare the pairwise correlations of the residual returns to that of the original returns. What do you notice?



In [19]:
ret.corr()

Ticker,AAPL,AMZN,GOOGL,META,NFLX,QQQ
Ticker,,,,,,
AAPL,1.000000,0.735558,0.655401,0.721762,0.487017,0.813980
AMZN,0.735558,1.000000,0.800036,0.869390,0.678042,0.899354
GOOGL,0.655401,0.800036,1.000000,0.715180,0.518240,0.814480
META,0.721762,0.869390,0.715180,1.000000,0.686227,0.866250
NFLX,0.487017,0.678042,0.518240,0.686227,1.000000,0.726044
QQQ,0.813980,0.899354,0.814480,0.866250,0.726044,1.000000


In [21]:
resid.corr()

Ticker,AAPL,AMZN,GOOGL,META,NFLX,QQQ
Ticker,,,,,,
AAPL,NaN,NaN,NaN,NaN,NaN,NaN
AMZN,NaN,NaN,NaN,NaN,NaN,NaN
GOOGL,NaN,NaN,NaN,NaN,NaN,NaN
META,NaN,NaN,NaN,NaN,NaN,NaN
NFLX,NaN,NaN,NaN,NaN,NaN,NaN
QQQ,NaN,NaN,NaN,NaN,NaN,NaN



6. Compute the information ratio for each of these stocks and compare that to the sharpe ratio

In [22]:
#  The IRs are in general lower than the SRs, suggesting most of the performance of these stocks was driven by their exposure to QQQ. AAPL performed the best looking at IR or SR and FB performed the worst.

df = {}
df['IR'] = resid.mean() / resid.std() * np.sqrt(252)
df['SR'] = ret.mean() / ret.std() * np.sqrt(252)
df = pd.DataFrame(df).drop('QQQ')
df

,IR,SR
Ticker,,
AAPL,NaN,-1.329308
AMZN,NaN,-1.314680
GOOGL,NaN,-1.573008
META,NaN,-0.492594
NFLX,NaN,0.510958
